In [205]:
import os
import pandas as pd
from tqdm import tqdm
import pandas as pd
import librosa
import json

In [162]:
df=pd.read_csv('metadata.csv')

In [165]:
df

,File_name,Covid_status,Gender,Nationality
0,ABZpUgKz_cough,n,f,I
1,ACKYAaOu_cough,n,m,I
2,AEthVqjZ_cough,n,m,I
3,AFuFLsFC_cough,n,m,I
4,AHGDPacO_cough,p,f,I
...,...,...,...,...
1035,zuEzdVVo_cough,p,m,I
1036,zwWWpDNo_cough,n,m,I
1037,zwYEGgem_cough,n,m,I
1038,zzOYyYyA_cough,n,m,I


In [167]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [192]:
listOfPositions = getIndexes(df, 'zzTJVzBg_cough')

In [193]:
df.Covid_status[listOfPositions[0][0]]

'n'

In [131]:
train_path=r'C:\Users\Saranga\Desktop\COVID cough\Folds!!!!!!!!!\LISTS\train_folds'

In [132]:
train_dict={1:[],2:[],3:[],4:[],5:[]}

In [133]:
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(train_path)):
    print('Processing ', dirpath.split("/")[-1])
    j=1
    for f in filenames:
        fpath=os.path.join(dirpath,f)
        count=0
        file=open(fpath,'r')
        while True:
            count += 1

            # Get next line from file
            line = file.readline()

            # if line is empty
            # end of file is reached
            
            if not line:
                j+=1
                break
            train_dict[j].append(line.strip())
            #print("{}".format(line.strip()))
            #print(i)
        
        file.close()
        

Processing  C:\Users\Saranga\Desktop\COVID cough\Folds!!!!!!!!!\LISTS\train_folds


In [134]:
val_path=r'C:\Users\Saranga\Desktop\COVID cough\Folds!!!!!!!!!\LISTS\val_folds'

In [135]:
val_dict={1:[],2:[],3:[],4:[],5:[]}

In [136]:
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(val_path)):
    print('Processing ', dirpath.split("/")[-1])
    j=1
    for f in filenames:
        fpath=os.path.join(dirpath,f)
        count=0
        file=open(fpath,'r')
        while True:
            count += 1

            # Get next line from file
            line = file.readline()

            # if line is empty
            # end of file is reached
            
            if not line:
                j+=1
                break
            val_dict[j].append(line.strip())
            #print("{}".format(line.strip()))
            #print(i)
        
        file.close()
        

Processing  C:\Users\Saranga\Desktop\COVID cough\Folds!!!!!!!!!\LISTS\val_folds


In [149]:
len(val_dict[1])

218

In [219]:
data_path=r'C:\Users\Saranga\Desktop\COVID cough\DiCOVA_Train_Val_Data_Release\AUDIO'
train_fold_dicts=[]
val_fold_dicts=[]

In [220]:
for key in train_dict.keys():
    
    data={
     'mfcc':[],
    'label':[]
     }

    for i,name in tqdm(enumerate(train_dict[key])):
        listOfPositions = getIndexes(df, name)
        label=df.Covid_status[listOfPositions[0][0]]
        if label=="n":
            label=0
        else:
            label=1
        
        fpath=os.path.join(data_path,name+'.flac')
        
        x,sr=librosa.load(fpath,22050)
        x=librosa.util.fix_length(x,154350) #7 seconds
        mfcc=librosa.feature.mfcc(x, n_mfcc=15, n_fft=2048, hop_length=512)
        mfcc=mfcc.T
        
        data['mfcc'].append(mfcc.tolist())
        data['label'].append(label)
        
    train_fold_dicts.append(data)
    
    
        
        
        

822it [02:26,  5.61it/s]
822it [02:27,  5.59it/s]
822it [02:28,  5.54it/s]
822it [03:17,  4.17it/s]
822it [03:33,  3.85it/s]


In [221]:
for i in tqdm(range(0,len(train_fold_dicts))):
    json_path=str(i+1)+'train_mfcc15.json'
    with open(json_path,"w") as fp:
        json.dump(train_fold_dicts[i],fp, indent=4)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:58<00:00, 11.72s/it]


In [222]:
type(train_fold_dicts[1])

dict

In [235]:
len(train_fold_dicts[1]['mfcc'][0][1])

15

In [244]:
for key in val_dict.keys():
    
    data={
     'mfcc':[],
    'label':[]
     }

    for i,name in tqdm(enumerate(val_dict[key])):
        listOfPositions = getIndexes(df, name)
        label=df.Covid_status[listOfPositions[0][0]]
        if label=="n":
            label=0
        else:
            label=1
        
        fpath=os.path.join(data_path,name+'.flac')
        
        x,sr=librosa.load(fpath,22050)
        x=librosa.util.fix_length(x,154350) #7 seconds
        mfcc=librosa.feature.mfcc(x, n_mfcc=15, n_fft=2048, hop_length=512)
        mfcc=mfcc.T
        
        data['mfcc'].append(mfcc.tolist())
        data['label'].append(label)
        
    val_fold_dicts.append(data)
    
    
        
        
        

218it [00:38,  5.67it/s]
218it [00:40,  5.45it/s]
218it [00:39,  5.52it/s]
218it [00:40,  5.40it/s]
218it [00:41,  5.26it/s]


In [245]:
for i in tqdm(range(0,len(val_fold_dicts))):
    json_path=str(i+1)+'val_mfcc15.json'
    with open(json_path,"w") as fp:
        json.dump(val_fold_dicts[i],fp, indent=4)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.45s/it]


10